In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 5022, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 5022 (delta 179), reused 154 (delta 131), pack-reused 4811
Receiving objects: 100% (5022/5022), 2.79 GiB | 26.61 MiB/s, done.
Resolving deltas: 100% (4005/4005), done.
Updating files: 100% (458/458), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 22.3 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv


In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender, SLIMElasticNetRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_SVDpp_Cython, MatrixFactorization_AsySVD_Cython, MatrixFactorization_WARP_Cython
from Recommenders.MatrixFactorization.PyTorch.MF_MSE_PyTorch import MF_MSE_PyTorch

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_slim_elastic_recall.db',
    'copy_prev_best_params': False,
    'save_github': True
}

In [10]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_slim_elastic_recall.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [11]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [12]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [13]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [14]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[50])

def objective_function_slim(trial):           
    params = {
        'l1_ratio': trial.suggest_float('l1_ratio', 1e-5, 1, log=True),
        'alpha': trial.suggest_float('alpha', 1e-5, 100, log=True),
        'topK': trial.suggest_int('topK', 5, 3000, log=True),
        'positive_only': trial.suggest_categorical("positive_only", [True, False]),
    }
        
    recommender_instance = SLIMElasticNetRecommender(URM_train)
    recommender_instance.fit(**params)
        
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    return result_df.loc[50, "RECALL"]

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions


In [15]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_slim',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_function_slim, n_trials=30)

[I 2023-12-16 16:17:42,285] Using an existing study with name 'hyperparameters_tuning_slim' instead of creating a new one.


SLIMElasticNetRecommender: URM Detected 627 ( 4.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 247 ( 1.1%) items with no interactions.
SLIMElasticNetRecommender: Processed 7791 (34.9%) in 5.00 min. Items per second: 25.96
SLIMElasticNetRecommender: Processed 16367 (73.2%) in 10.00 min. Items per second: 27.27
SLIMElasticNetRecommender: Processed 22347 (100.0%) in 13.66 min. Items per second: 27.27
EvaluatorHoldout: Processed 10456 (100.0%) in 14.34 sec. Users per second: 729


[I 2023-12-16 16:31:36,877] Trial 210 finished with value: 0.27201458088762376 and parameters: {'l1_ratio': 8.377904949729038e-05, 'alpha': 0.004774211721016447, 'topK': 2114, 'positive_only': True}. Best is trial 112 with value: 0.2761653571343395.


SLIMElasticNetRecommender: URM Detected 627 ( 4.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 247 ( 1.1%) items with no interactions.
SLIMElasticNetRecommender: Processed 8437 (37.8%) in 5.00 min. Items per second: 28.12
SLIMElasticNetRecommender: Processed 17637 (78.9%) in 10.00 min. Items per second: 29.39
SLIMElasticNetRecommender: Processed 22347 (100.0%) in 12.78 min. Items per second: 29.14
EvaluatorHoldout: Processed 10456 (100.0%) in 15.37 sec. Users per second: 680


[I 2023-12-16 16:44:39,908] Trial 211 finished with value: 0.27543027074374454 and parameters: {'l1_ratio': 0.00020856332205483973, 'alpha': 0.010245346069288607, 'topK': 2654, 'positive_only': True}. Best is trial 112 with value: 0.2761653571343395.


...

[I 2023-12-16 22:57:55,744] Trial 237 finished with value: 0.27572056372944437 and parameters: {'l1_ratio': 0.00014094891663473858, 'alpha': 0.01083923138016588, 'topK': 1261, 'positive_only': True}. Best is trial 112 with value: 0.2761653571343395.


SLIMElasticNetRecommender: URM Detected 627 ( 4.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 247 ( 1.1%) items with no interactions.
SLIMElasticNetRecommender: Processed 8444 (37.8%) in 5.00 min. Items per second: 28.14
SLIMElasticNetRecommender: Processed 17506 (78.3%) in 10.00 min. Items per second: 29.17
SLIMElasticNetRecommender: Processed 22347 (100.0%) in 12.88 min. Items per second: 28.90
EvaluatorHoldout: Processed 10456 (100.0%) in 15.28 sec. Users per second: 684


[I 2023-12-16 23:11:05,130] Trial 238 finished with value: 0.2755801176610853 and parameters: {'l1_ratio': 0.0002000132787557398, 'alpha': 0.009404330089710394, 'topK': 1286, 'positive_only': True}. Best is trial 112 with value: 0.2761653571343395.


SLIMElasticNetRecommender: URM Detected 627 ( 4.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 247 ( 1.1%) items with no interactions.
SLIMElasticNetRecommender: Processed 8357 (37.4%) in 5.00 min. Items per second: 27.85
SLIMElasticNetRecommender: Processed 17634 (78.9%) in 10.00 min. Items per second: 29.39
SLIMElasticNetRecommender: Processed 22347 (100.0%) in 12.76 min. Items per second: 29.19
EvaluatorHoldout: Processed 10456 (100.0%) in 16.69 sec. Users per second: 627


[I 2023-12-16 23:24:08,220] Trial 239 finished with value: 0.2656869012912718 and parameters: {'l1_ratio': 0.00015252301035301825, 'alpha': 0.034007234616996125, 'topK': 1079, 'positive_only': True}. Best is trial 112 with value: 0.2761653571343395.


In [16]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [17]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [18]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [19]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_slim_recall@50.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            '/kaggle/working/best_params_slim_recall@50.json', 
            'tuning_results/best_params_slim_recall@50.json', 
            'SLIM Elastic Net tuning results with recall@50(from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_slim_recall@50.json', 
        '/kaggle/working/best_params_slim_recall@50.json'
    )

In [20]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        'tuning_results/tuning_slim_elastic_recall.db',
        'Tuning db updated results (from kaggle notebook)'
    )